In [ ]:
from threeML import *
import numpy as np
from threeML.plugins import *
from skylab.datasets import Datasets
from astropy import units as u

import os, sys, glob, abc

def read(filelist):
    data = []
    for f in sorted(filelist):
        x = np.load(f)
        if len(data) == 0: data = x.copy()
        else: data = np.concatenate([data, x])
    return data

# Where is the dataset stored?
dataset = "/data/i3store/users/mjlarson/student_data"

# Read in all of the data files
data_files = dataset + "/IC86_*exp.npy"
exp = read(glob.glob(data_files))

# Read in all of the MC files 
sim_files = dataset + "/IC86*MC*npy"
mc = read(glob.glob(sim_files))

# Set the angular error floor to 0.2 degrees
exp['angErr'][exp['angErr']<np.deg2rad(0.2)] = np.deg2rad(0.2)
mc['angErr'][mc['angErr']<np.deg2rad(0.2)] = np.deg2rad(0.2)

exp['ra']=np.random.uniform(0,2*np.pi,size=len(exp))
grl = np.load("/data/i3store/users/mjlarson/student_data/GRL/IC86_2012_exp.npy")
livetime = np.sum(grl['livetime'])


In [6]:
from imp import reload
reload(IceCubeLike)

import warnings
warnings.filterwarnings("ignore")

IceCube=IceCubeLike.IceCubeLike("test",exp,mc,livetime,sinDec_bins=np.linspace(-1,1,30))


In [7]:
source1_sp = Powerlaw()
#source1_sp._set_units(u.GeV,u.cm**-2 / u.s / u.GeV)
source1_sp.K=1e-13
source1_neutrino = Powerlaw()
source1_neutrino.K=1e-10
#source1_sp._set_units(u.GeV,u.cm**-2 / u.s / u.GeV)
source1 = PointSource("source1", ra=83.63, dec=22.0145, spectral_shape=source1_sp)
source1_nu = ParticleSource("source1_nu",distribution_shape=source1_neutrino)
model = Model(source1,source1_nu)
model.source1.spectrum.main.Powerlaw.K.fix = True
model.source1.spectrum.main.Powerlaw.index.fix = True
IceCubedata = DataList(IceCube)
jl = JointLikelihood(model, IceCubedata)

In [8]:
data = np.random.choice(exp, 100).copy()
jl._data_list['test'].llh_model.update_data(data,10000/3600/24)

In [9]:
best_fit_parameters, likelihood_values = jl.fit()

(4.917044499098152e-05, -9.834091416417109e-05)
(4.939740904788845e-05, -9.879484250520143e-05)
(4.894451768687511e-05, -9.78890593221156e-05)
(5.148752625507201e-05, -0.00010297507901688864)
(4.695705656405809e-05, -9.391413518030272e-05)
(4.937199049553019e-05, -9.874400536779802e-05)
(4.8969753498954676e-05, -9.793953097370294e-05)


RuntimeError: exception was raised in user function
User function arguments:
        source1_nu_spectrum_main_Powerlaw_K = -10.000000
    source1_nu_spectrum_main_Powerlaw_index = -1.996000
Original python exception in user function:
KeyboardInterrupt: 
  File "/data/disk01/home/jasonfan/threeML/threeML/classicMLE/joint_likelihood.py", line 745, in minus_log_like_profile
    this_log_like = dataset.inner_fit()
  File "/data/disk01/home/jasonfan/threeML/threeML/plugins/IceCubeLike.py", line 129, in inner_fit
    return self.get_log_like()
  File "/data/disk01/home/jasonfan/threeML/threeML/plugins/IceCubeLike.py", line 123, in get_log_like
    self.update_model()
  File "/data/disk01/home/jasonfan/threeML/threeML/plugins/IceCubeLike.py", line 114, in update_model
    self.llh_model.updata_spectrum(self.spectrum)
  File "/data/disk01/home/jasonfan/umd_icecube_analysis_tutorial/mla/mla/core.py", line 110, in updata_spectrum
    self.update_energy_histogram()
  File "/data/disk01/home/jason

In [13]:
from mla import injection
from imp import reload
reload(injection)
reload(IceCubeLike)
from mla.spectral import *
sp= PowerLaw(1,1e-15,2)
dec = 22.0145*np.pi/180
ra = 83.63*np.pi/180
inj = injection.PSinjector(sp,mc)
inj.set_source_location(ra,dec)
sample=inj.sample_nevents(100)
IceCube=IceCubeLike.IceCubeLike("test",exp,mc,livetime,sinDec_bins=np.linspace(-1,1,30))
source1_sp = Powerlaw()
#source1_sp._set_units(u.GeV,u.cm**-2 / u.s / u.GeV)
source1_sp.K=1e-13
source1_neutrino = Powerlaw()
source1_neutrino.K=1e-5
#source1_sp._set_units(u.GeV,u.cm**-2 / u.s / u.GeV)
source1 = PointSource("source1", ra=83.63, dec=22.0145, spectral_shape=source1_sp)
source1_nu = ParticleSource("source1_nu",distribution_shape=source1_neutrino)
model = Model(source1,source1_nu)
IceCube.add_injection(sample)
model.source1.spectrum.main.Powerlaw.K.fix = True
model.source1.spectrum.main.Powerlaw.index.fix = True
IceCubedata = DataList(IceCube)
jl = JointLikelihood(model, IceCubedata)
data=np.load("testdata.npy")

In [14]:
jl._data_list['test'].llh_model.update_data(data,10000/3600/24)
#jl._data_list['test'].llh_model.add_injection(sample)

In [15]:

x=np.linspace(0,jl._data_list['test'].llh_model.N)
temp=[]
for i in x:
    temp.append(-jl._data_list['test'].llh_model.eval_llh_ns)

(4.917044499098152, -10.414016610292125)
(4.92837941321164, -10.439470984248207)
(4.905735174371189, -10.38862687420419)
(4.92025816313321, -10.4212326789354)
(4.913832895348487, -10.406805745853783)
(4.937199049553018, -10.45928187846076)
(4.896975349895467, -10.368965755702453)
(4.9201788350019005, -10.421054545932911)
(4.913912236035355, -10.406983878845793)
(0.8455421178065752, -1.7071736880148616)
(1.0024083424545795, -2.0274833165276087)
(0.5676491983081287, -1.1425197589475926)
(0.2931025186222823, -0.5881224585297569)


RuntimeError: exception was raised in user function
User function arguments:
        source1_nu_spectrum_main_Powerlaw_K = -5.739770
    source1_nu_spectrum_main_Powerlaw_index = -2.155848
Original python exception in user function:
KeyboardInterrupt: 
  File "/data/disk01/home/jasonfan/threeML/threeML/classicMLE/joint_likelihood.py", line 745, in minus_log_like_profile
    this_log_like = dataset.inner_fit()
  File "/data/disk01/home/jasonfan/threeML/threeML/plugins/IceCubeLike.py", line 129, in inner_fit
    return self.get_log_like()
  File "/data/disk01/home/jasonfan/threeML/threeML/plugins/IceCubeLike.py", line 123, in get_log_like
    self.update_model()
  File "/data/disk01/home/jasonfan/threeML/threeML/plugins/IceCubeLike.py", line 114, in update_model
    self.llh_model.updata_spectrum(self.spectrum)
  File "/data/disk01/home/jasonfan/umd_icecube_analysis_tutorial/mla/mla/core.py", line 110, in updata_spectrum
    self.update_energy_histogram()
  File "/data/disk01/home/jasonf